# Entity Service Permutation Output

This notebook demonstrates generating CLKs from PII, creating a new mapping on the entity service, and how to retrieve the results. The output type is permutation and unencrypted mask.

The sections are usually run on different companies - but for illustration all is carried out in this one file. The participants providing data are *Alice* and *Bob*, and the analyst acting the integration authority.

### Who learns what?

Alice and Bob will both generate and upload their CLKs. After the linkage has been carried out they will be able to retrieve a `permutation` - a reordering of their respective data sets such that shared entities line up.

The analyst - who creates the linkage project - learns the `mask`. The mask is a binary vector that indicates which rows in the permuted data sets are aligned. Note this reveals how many entities are shared. 

### Steps

* Check connection to Entity Service
* Data preparation
  * Write CSV files with PII
  * Create a Linkage Schema
  
* Create Linkage Project
* Generate CLKs from PII
* Upload the PII
* Retrieve and analyse results

## Check Connection

If you are connecting to a custom entity service, change the address here.

In [1]:
url = 'https://es.data61.xyz'

In [2]:
!clkutil status --server "{url}"

Connecting to Entity Matching Server: https://es.data61.xyz
Response: 200
Status: ok
{"number_mappings": 3434, "rate": 3347647, "status": "ok"}


## Data preparation

Following the clkhash tutorial we will use a dataset from the `recordlinkage` library. We will just write both datasets out to temporary CSV files.

If you are following along yourself you may have to adjust the file names in all the `!clkutil` commands.

In [3]:
from tempfile import NamedTemporaryFile
from recordlinkage.datasets import load_febrl4

In [4]:
dfA, dfB = load_febrl4()

a_csv = NamedTemporaryFile('w')
a_clks = NamedTemporaryFile('w', suffix='.json')
dfA.to_csv(a_csv)
a_csv.seek(0)

b_csv = NamedTemporaryFile('w')
b_clks = NamedTemporaryFile('w', suffix='.json')
dfB.to_csv(b_csv)
b_csv.seek(0)

dfA.head()
print("Datasets written to {} and {}".format(a_csv.name, b_csv.name))

Datasets written to /tmp/tmpc57_lneo and /tmp/tmpv20xs4ku


In [5]:
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


## Schema Preparation

The linkage schema must be agreed on by the two parties. For this tutorial let's say that the only fields in common were Surname, First Name, Postcode, State, and Date of birth. We create a schema that ignores all other columns when creating CLKs:

In [6]:
column_metadata = [
    'INDEX',
    'NAME Surname',
    'NAME First Name',
    'INDEX',
    'INDEX',
    'INDEX',
    'INDEX',
    'ADDRESS POSTCODE',
    'ADDRESS Place Name',
    'DOB YYYY/MM/DD',
    'INDEX'
]

schema = NamedTemporaryFile("wt", suffix='.yaml')
for col in column_metadata:
    print('- identifier: "{}"'.format(col), file=schema)

schema.seek(0)
print("Schema written to", schema.name)

Schema written to /tmp/tmplqqn1v9k.yaml


## Create Linkage Project

The analyst carrying out the linkage starts by creating a linkage project of the desired output type with the Entity Service.


In [7]:
creds = NamedTemporaryFile('wt')
print("Credentials will be saved in", creds.name)

!clkutil create --schema "{schema.name}" --output "{creds.name}" --type "permutation_unencrypted_mask" --server "{url}" --threshold 0.85
creds.seek(0)

import json
with open(creds.name, 'r') as f:
    credentials = json.load(f)

mid = credentials['resource_id']
credentials

Credentials will be saved in /tmp/tmpf8dx1rwk
Entity Matching Server: https://es.data61.xyz
Checking server status
Server Status: ok
Schema: [{"identifier": "INDEX"}, {"identifier": "NAME Surname"}, {"identifier": "NAME First Name"}, {"identifier": "INDEX"}, {"identifier": "INDEX"}, {"identifier": "INDEX"}, {"identifier": "INDEX"}, {"identifier": "ADDRESS POSTCODE"}, {"identifier": "ADDRESS Place Name"}, {"identifier": "DOB YYYY/MM/DD"}, {"identifier": "INDEX"}]
Type: permutation_unencrypted_mask
Creating new mapping
Mapping created


{'resource_id': '4e82ce916a1ee6c3012f0a9a1c5c1ed46b62b08829583891',
 'result_token': 'bb9f11216cdd39538f82576458cfb75ce984d48a12bed9fc',
 'update_tokens': ['68d74f19523b09f7d600649c6b62454d51d7b41bd13c571b',
  'cc29bcbcb4a296a596f91f4f99f514c116224e2ed69922c8']}

**Note:** the analyst will need to pass on the `resource_id` (the id of the linkage project) and one of the two `update_tokens` to each data provider.

## Hash and Upload

At the moment both data providers have *raw* personally identiy information. We first have to generate CLKs from the raw entity information. Please see [clkhash](https://clkhash.readthedocs.io/) documentation for further details on this.

In [8]:
!clkutil hash --schema "{schema.name}" "{a_csv.name}" horse staple "{a_clks.name}"
!clkutil hash --schema "{schema.name}" "{b_csv.name}" horse staple "{b_clks.name}"

generating CLKs: 100%|█| 5.00K/5.00K [00:01<00:00, 1.95Kclk/s, mean=651, std=36.1]
CLK data written to /tmp/tmpyyyzbtl6.json
generating CLKs: 100%|█| 5.00K/5.00K [00:01<00:00, 1.07Kclk/s, mean=647, std=40.5]
CLK data written to /tmp/tmpf0rqxh20.json


Now the two clients can upload their data providing the appropriate *upload tokens*. As with all commands in `clkhash` we can output help:

In [9]:
!clkutil upload --help

Usage: clkutil upload [OPTIONS] INPUT

  Upload CLK data to entity matching server.

  Given a json file containing hashed clk data as INPUT, upload to the
  entity resolution service.

  Use "-" to read from stdin.

Options:
  --mapping TEXT         Server identifier of the mapping
  --apikey TEXT          Authentication API key for the server.
  --server TEXT          Server address including protocol
  -o, --output FILENAME
  -v, --verbose          Script is more talkative
  --help                 Show this message and exit.


### Alice uploads her data

In [10]:
with NamedTemporaryFile('wt') as f:
    !clkutil upload \
        --mapping="{credentials['resource_id']}" \
        --apikey="{credentials['update_tokens'][0]}" \
        --server "{url}" \
        --output "{f.name}" \
        "{a_clks.name}"
    res = json.load(open(f.name))
    alice_receipt_token = res['receipt-token']

Uploading CLK data from /tmp/tmpyyyzbtl6.json
To Entity Matching Server: https://es.data61.xyz
Mapping ID: 4e82ce916a1ee6c3012f0a9a1c5c1ed46b62b08829583891
Checking server status
Status: ok
Uploading CLK data to the server


Every upload gets a receipt token. In some operating modes this receipt is required to access the results.

### Bob uploads his data

In [11]:
with NamedTemporaryFile('wt') as f:
    !clkutil upload \
        --mapping="{credentials['resource_id']}" \
        --apikey="{credentials['update_tokens'][1]}" \
        --server "{url}" \
        --output "{f.name}" \
        "{b_clks.name}"
    
    bob_receipt_token = json.load(open(f.name))['receipt-token']

Uploading CLK data from /tmp/tmpf0rqxh20.json
To Entity Matching Server: https://es.data61.xyz
Mapping ID: 4e82ce916a1ee6c3012f0a9a1c5c1ed46b62b08829583891
Checking server status
Status: ok
Uploading CLK data to the server


## Results

Now after some delay (depending on the size) we can fetch the mask.
This can be done with clkutil:

    !clkutil results \
        --mapping="{credentials['resource_id']}" \
        --apikey="{credentials['result_token']}" --output results.txt
        
However for this tutorial we are going to use the Python `requests` library:

In [12]:
import requests
import json
import time

In [13]:
# this cheeky cell is here to impose a small delay to allow the Entity Service to carry out the linkage when running the notebook automatically
time.sleep(2)
requests.get('{}/api/v1/mappings/{}/status'.format(url, mid), headers={'Authorization': credentials['result_token']}).json()

{'ready': True,
 'threshold': 0.85,
 'time_added': '2018-03-27T10:24:40.346932',
 'time_completed': '2018-03-27T10:24:50.005947',
 'time_started': '2018-03-27T10:24:49.338349'}

In [14]:
result = requests.get('{}/api/v1/mappings/{}'.format(url, mid), headers={'Authorization': credentials['result_token']})
while result.status_code != 200:
    print(result.json())
    result = requests.get('{}/api/v1/mappings/{}'.format(url, mid), headers={'Authorization': credentials['result_token']})
else:
    results = result.json()

In [15]:
mask = results['mask']

This mask is a boolean array that specifies where rows of permuted data line up.

In [16]:
print(mask[:10])

[1, 1, 0, 1, 1, 1, 1, 0, 1, 0]


The number of 1s in the mask will tell us how many matches were found.

In [17]:
sum([1 for m in mask if m == 1])

4417

We also use `requests` to fetch the permutations for each data provider:

In [18]:
alice_res = requests.get('{}/api/v1/mappings/{}'.format(url, mid), headers={'Authorization': alice_receipt_token}).json()
bob_res = requests.get('{}/api/v1/mappings/{}'.format(url, mid), headers={'Authorization': bob_receipt_token}).json()

Now Alice and Bob both have a new permutation - a new ordering for their data.

In [19]:
alice_permutation = alice_res['permutation']
alice_permutation[:10]

[308, 4653, 3602, 439, 2287, 389, 4577, 1125, 4878, 158]

This permutation says the first row of Alice's data should be moved to position 308.

In [20]:
bob_permutation = bob_res['permutation']
bob_permutation[:10]

[1651, 4849, 4530, 350, 4897, 3015, 2768, 4090, 4751, 1797]

In [21]:
def reorder(items, order):
    """
    Assume order is a list of new index
    """
    neworder = items.copy()
    for item, newpos in zip(items, order):
        neworder[newpos] = item
    
    return neworder

In [22]:
with open(a_csv.name, 'r') as f:
    alice_raw = f.readlines()[1:]
    alice_reordered = reorder(alice_raw, alice_permutation)

with open(b_csv.name, 'r') as f:
    bob_raw = f.readlines()[1:]
    bob_reordered = reorder(bob_raw, bob_permutation)

Now that the two data sets have been permuted, the mask reveals where the rows line up, and where they don't.

In [23]:
alice_reordered[:10]

['rec-2198-org,matilda,bergsma,73,macrobertson street,derry lodge,greensborough,2076,wa,19880213,5520743\n',
 'rec-4413-org,cameron,white,178,carlile street,laureldale,alice springs,6108,nsw,19470605,1655664\n',
 'rec-2064-org,jasper,miles,20,narryer close,moondah,nambour,4113,vic,19160130,3809763\n',
 'rec-4511-org,levi,oaks,38,courtice close,sunshine farm,findon,6050,wa,19101128,9317947\n',
 'rec-3659-org,alexander,vincent,51,gallagher street,top end,belmont,2333,qld,19150726,3732438\n',
 'rec-476-org,kyle,armiento,266,jaeger circuit,kookaburra village,springwood,7307,vic,19570728,3494623\n',
 'rec-998-org,bailey,green,76,jackie howe crescent,glengara village,murgon,3340,nsw,19960416,9882349\n',
 'rec-139-org,holly,rieman,3,heagney crescent,winter park,devonport,2546,qld,19751221,5260793\n',
 'rec-3071-org,charlie,walpole,43,lampard circuit,the meadows,alberton,2750,sa,19491212,5982948\n',
 'rec-2962-org,ethan,ryan,20,eaglemont retreat,melody cottage,camperdown,2615,nsw,19320524,2653

In [24]:
bob_reordered[:10]

['rec-2198-dup-0,matilda,bergsma,73,macrobertson street,derrym lodge,greensborough,2076,wa,19880213,5520743\n',
 'rec-4413-dup-0,cameron,white,178,carliles treet,laureldale,harris park,6180,nsw,19470605,1655664\n',
 'rec-563-dup-0,sybella,meaney,82,julius street,east end,seafroth,3690,sa,,7466921\n',
 'rec-4511-dup-0,levi,oaks,38,courtice close,sunshin e farm,findon,6050,wa,19101128,9317947\n',
 'rec-3659-dup-0,ale xander,vincent,51,gallagher street,top end,belmont,2333,qld,19150726,3732438\n',
 'rec-476-dup-0,klye,armient o,266,jaegerc ircuit,kookabura village,elwood,7307,vic,19570728,3494623\n',
 'rec-998-dup-0,bailey,gree n,76,jackie howe crescent,glengara billage,murgon,3340,nsw,19960416,1468056\n',
 'rec-425-dup-0,psorakis,shantal,61,barcoo place,stonyridge,hawtnhorn,2324,vic,19850722,6633125\n',
 'rec-3071-dup-0,charlie,wakpbole,43,lampard circuit,the meadows,albertno,2750,sa,19260710,5982948\n',
 'rec-4496-dup-0,warnock,hark,,jansz cr escent,brentwood vlge,broadmeadows,2486,,193

## Accuracy

To compute how well the matching went we will use the first index as our reference.

For example in `rec-1396-org` is the original record which has a match in `rec-1396-dup-0`. To satisfy ourselves we can preview the first few supposed matches:

In [25]:
for i, m in enumerate(mask[:10]):
    if m:
        entity_a = alice_reordered[i].split(',')
        entity_b = bob_reordered[i].split(',')
        name_a = ' '.join(entity_a[1:3]).title()
        name_b = ' '.join(entity_b[1:3]).title()
        
        print("{} ({})".format(name_a, entity_a[0]), '=?', "{} ({})".format(name_b, entity_b[0]))

Matilda Bergsma (rec-2198-org) =? Matilda Bergsma (rec-2198-dup-0)
Cameron White (rec-4413-org) =? Cameron White (rec-4413-dup-0)
Levi Oaks (rec-4511-org) =? Levi Oaks (rec-4511-dup-0)
Alexander Vincent (rec-3659-org) =? Ale Xander Vincent (rec-3659-dup-0)
Kyle Armiento (rec-476-org) =? Klye Armient O (rec-476-dup-0)
Bailey Green (rec-998-org) =? Bailey Gree N (rec-998-dup-0)
Charlie Walpole (rec-3071-org) =? Charlie Wakpbole (rec-3071-dup-0)


### Metrics

**Precision**: The percentage of actual matches out of all found matches. (`tp/(tp+fp)`)

**Recall**: How many of the actual matches have we found? (`tp/(tp+fn)`)

In [26]:
tp = 0
fp = 0

for i, m in enumerate(mask):
    if m:
        entity_a = alice_reordered[i].split(',')
        entity_b = bob_reordered[i].split(',')
        if entity_a[0].split('-')[1] == entity_b[0].split('-')[1]:
            tp += 1
        else:
            fp += 1
            #print('False positive:',' '.join(entity_a[1:3]).title(), '?', ' '.join(entity_b[1:3]).title(), entity_a[-1] == entity_b[-1])

print("Found {} correct matches out of 5000. Incorrectly linked {} matches.".format(tp, fp))
precision = tp/(tp+fp)
recall = tp/5000

print("Precision: {:.1f}%".format(100*precision))
print("Recall: {:.1f}%".format(100*recall))

Found 4416 correct matches out of 5000. Incorrectly linked 1 matches.
Precision: 100.0%
Recall: 88.3%
